In [92]:
import requests
import os
import json
from api_keys import BEARER_TOKEN
from tqdm import tqdm
import os
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Funções (Análise Exploratória)

### Função para plotar bar plot com a contagem de tokens

In [65]:
def plot_bar_count_words(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = CountVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para plotar bar plot com tf-idf

In [104]:
def plot_bar_tf_idf(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = TfidfVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    "MAX":data_vect.std(axis=0)
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para contagem de tokens

In [66]:
def calculate_number_words(text):

    quantity_of_words = text.split(" ")

    quantity_of_words = [i for i in quantity_of_words if i!=""]

    quantity_of_words = len(quantity_of_words)

    return quantity_of_words


### Função para contagem de diferentes tokens

In [67]:
def calculate_number_diferent_words(text):

    quantity_of_diferent_words = text.split(" ")

    quantity_of_diferent_words = [i for i in quantity_of_diferent_words if i!=""]

    quantity_of_diferent_words = set(quantity_of_diferent_words)

    quantity_of_diferent_words = list(quantity_of_diferent_words)

    quantity_of_diferent_words = len(quantity_of_diferent_words)

    return quantity_of_diferent_words


### Função para criar textos sem repetição de palavras para ser utilizado na análise exploratória 

In [68]:
def convert_text_to_no_repeat_words(text):

    text_with_no_repeat_words = text.split(" ")

    text_with_no_repeat_words = [i for i in text_with_no_repeat_words if i!=""]

    text_with_no_repeat_words = set(text_with_no_repeat_words)

    text_with_no_repeat_words = list(text_with_no_repeat_words)

    text_with_no_repeat_words = " ".join(text_with_no_repeat_words)

    return text_with_no_repeat_words

### Função para o pré-processamento do texto 

In [69]:
def text_cleaner(text):
    
    nltk_stopwords = stopwords.words('portuguese')

    collection_text = [ {"text" : text}]
    text = pd.DataFrame(collection_text)

    text['text'] = text['text'].astype('str')
    text['text'] = text['text'].str.lower()
    text['text'] = text['text'].str.replace('\n',' ')
    text['text'] = text['text'].str.replace('\r',' ')
    text['text'] = text['text'].apply(lambda x: norm('NFKD', x).encode('ascii', 'ignore').decode())
    text['text'] = text['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
    text['text'] = text['text'].apply(lambda x: re.sub(r'\s+',' ',x))
    pat = r'\b(?:{})\b'.format('|'.join(nltk_stopwords))
    text['text'] = text['text'].str.replace(pat,'')
    text = text['text'].values[0]

    return text

# Funções (Extração de Tweets)

In [70]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_url(query = "@globoplay -is:retweet",until_id=None):
    
    #query = "@BBB -is:retweet"
    #"from:twitterdev -is:retweet"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    
    if until_id:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100&until_id={}".format(
            query,until_id
        )
        
    else:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=100".format(
            query
        )
            
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def extract_100_tweets(query = "@BBB -is:retweet",until_id=None):
    bearer_token = BEARER_TOKEN
    url = create_url(query,until_id)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    data_tweets = json.dumps(json_response, indent=4, sort_keys=True)
    return json_response

def extract_many_tweets(qnt_cycle=10,folder="data_tweets",start_from_id=None,query="@BBB"):
    
    
    oldest_id = None
    
    for i in tqdm(range(qnt_cycle)):
    
        
        if i == 0:
            
            #extract the 100 tweets first
            
            if start_from_id:
        
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=None)
            
            else:
                
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=start_from_id)
                
            
            df_data_tweets_temp = pd.DataFrame(data_tweets["data"])
            
            #get the current date
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            
            oldest_id = data_tweets['meta']['oldest_id']
            
            oldest_date = date_extraction
            
            df_data_tweets = df_data_tweets_temp.copy()
            
            # name file
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,date_extraction_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file,sep=",")
            
    
            
        else:
            
            
            #extract more 100 tweets older
            
            data_tweets_temp = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=oldest_id)
            
            df_data_tweets_temp = pd.DataFrame(data_tweets_temp["data"])
            
            
            #get the current date
            
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            oldest_id = data_tweets_temp['meta']['oldest_id']
            
            df_data_tweets = pd.concat([df_data_tweets,df_data_tweets_temp.copy()])
            
            date_extraction = datetime.now()
            
            df_data_tweets.reset_index(inplace=True,drop=True)
            
            
            # remove old files
            
            os.remove(name_file)
            
            
            # name file
            
            oldest_date_str = str(oldest_date).replace(".","-").replace(":","-").replace(" ","-")
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,oldest_date_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file.format(folder),sep=",")
            
            

    return df_data_tweets
    
    


# Extração de Tweets

In [97]:
data_tweets_final = extract_many_tweets(qnt_cycle=10,folder="data_tweets",query="rodolfo")#,start_from_id="1367600965277384706")

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


In [100]:
data_tweets_final.head(30)

id                                               text  \
0   1379630323110141952       Pra mim o Rodolfo não entendeu nada #BBB2021   
1   1379630322753662981                    RODOLFO ENTENDEU NADA AINDA pqp   
2   1379630322724315136  rodolfo além de racista é burro https://t.co/U...   
3   1379630322720108544  Já podia ter eliminado o Rodolfo no final do d...   
4   1379630322707468290  ta bom rodolfo na proxima vai aprender levando...   
5   1379630322669731843                             O Rodolfo é muito frio   
6   1379630322636193792  Caio saiu, se o Rodolfo saísse, ele iria deixa...   
7   1379630322590027781  @migaregis vai dizer jaja amg, mas já adianto ...   
8   1379630322573266947               Tô com pena do Rodolfo sim e foda-se   
9   1379630322304888832  Me julguem, mais até fiquei com dó do Rodolfo ...   
10  1379630322229346309      E o Rodolfo ainda não entendeu! É isso mesmo?   
11  1379630322090926081                   Se fosse Rodolfo pedia para sair   
12  1379630322069950471                           @idolshelpffxofi Rodolfo   
13  1379630321994436614           rodolfo a pessoa@mais sonsa que eu já vi   
14  1379630321772154880            Rodolfo com 57% https://t.co/y0bxCrx1yN   
15  1379630321419816962  Né possivi que o Rodolfo ainda não entendeu ve...   
16  1379630321294053376  Mds passaram uns 30min falando e explicando E ...   
17  1379630321080152073  O Rodolfo não se ajuda. Ele continua sem enten...   
18  1379630321046593538  Tadinho do Rodolfo, uma juntada dessas e depoi...   
19  1379630320950083588  Rodolfo não quer aprender gente, ponto final. ...   
20  1379630320702607362  Eu, se fosse o Tiago, pedia um quadrinho pra p...   
21  1379630320669097987  mano o duro que o rodolfo não entendeu absolut...   
22  1379630320434212864  @vittorfernando Gente que agonia do Rodolfo 🤦🏻‍♂️   
23  1379630320253865992     se o Rodolfo n for eliminado é o fim da picada   
24  1379630320111263747     Rodolfo jogou a última pá de terra na sua cova   
25  1379630319494692865  É triste saber que o Rodolfo pode ficar sendo ...   
26  1379630319310151682  o Rodolfo pedindo o contato da professora do f...   
27  1379630319083610120  teve aula exclusiva pro rodolfo no bbb e o car...   
28  1379630318613848065  Tiago fala e fala e Rodolfo não aprende nada f...   
29  1379630318429306885  O Rodolfo tem que sair.. não aceito esse macho...   

              date_extraction  
0  2021-04-07 00:01:27.111057  
1  2021-04-07 00:01:27.111057  
2  2021-04-07 00:01:27.111057  
3  2021-04-07 00:01:27.111057  
4  2021-04-07 00:01:27.111057  
5  2021-04-07 00:01:27.111057  
6  2021-04-07 00:01:27.111057  
7  2021-04-07 00:01:27.111057  
8  2021-04-07 00:01:27.111057  
9  2021-04-07 00:01:27.111057  
10 2021-04-07 00:01:27.111057  
11 2021-04-07 00:01:27.111057  
12 2021-04-07 00:01:27.111057  
13 2021-04-07 00:01:27.111057  
14 2021-04-07 00:01:27.111057  
15 2021-04-07 00:01:27.111057  
16 2021-04-07 00:01:27.111057  
17 2021-04-07 00:01:27.111057  
18 2021-04-07 00:01:27.111057  
19 2021-04-07 00:01:27.111057  
20 2021-04-07 00:01:27.111057  
21 2021-04-07 00:01:27.111057  
22 2021-04-07 00:01:27.111057  
23 2021-04-07 00:01:27.111057  
24 2021-04-07 00:01:27.111057  
25 2021-04-07 00:01:27.111057  
26 2021-04-07 00:01:27.111057  
27 2021-04-07 00:01:27.111057  
28 2021-04-07 00:01:27.111057  
29 2021-04-07 00:01:27.111057

# Criação de uma coluna com os textos sem repetição de palavras para ser utilizado na análise exploratória

In [73]:
data_tweets_final['text_unique_words'] = data_tweets_final['text'].apply(lambda x: convert_text_to_no_repeat_words(x))

# Calculo Número de tokens

In [74]:
data_tweets_final['number_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_words(x))

# Calculo Número de diferentes tokens

In [76]:
data_tweets_final['number_diferent_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_diferent_words(x))

# DF top 10 MEAN

In [60]:
df_report_mean = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [61]:
df_report_mean

MEAN     WORDS
1421  1.034  juliette
307   0.358     bbb21
2156  0.348       que
702   0.276        de
1818  0.254       não
536   0.254        co
1293  0.254     https
236   0.251    arthur
690   0.243        da
555   0.217       com

# DF top 10 SUM

In [62]:
df_report_sum = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='SUM',top=10,return_df=True)

In [63]:
df_report_sum

SUM     WORDS
1421  1034  juliette
307    358     bbb21
2156   348       que
702    276        de
1818   254       não
536    254        co
1293   254     https
236    251    arthur
690    243        da
555    217       com

# DF top 10 MEAN TF-IDF

In [105]:
df_report_tfidf_mean = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [106]:
df_report_tfidf_mean

MEAN     WORDS
1962  0.071392   rodolfo
1538  0.058325       não
1858  0.047692       que
703   0.036912        do
580   0.032307        de
810   0.031369  entendeu
2010  0.030431        se
745   0.027117       ele
269   0.026819     bbb21
1471  0.025393      nada

# DF top 10 MAX TF-IDF

In [107]:
df_report_tfidf_max = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MAX',top=10,return_df=True)

In [108]:
df_report_tfidf_max

MAX     WORDS
810   0.092226  entendeu
920   0.089776     falou
1471  0.086475      nada
1538  0.085212       não
1858  0.080693       que
326   0.079665     burro
1994  0.076654      sair
2287  0.076140       vai
703   0.073993        do
118   0.071560     ainda